### Just a quick example of code that runs through all images in validation set and calculates competition metric

In [7]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-t7xgu_42
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-t7xgu_42
     |████████████████████████████████| 49 kB 3.9 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
     |████████████████████████████████| 145 kB 55.9 MB/s 
     |████████████████████████████████| 130 kB 60.0 MB/s 
     |████████████████████████████████| 749 kB 60.0 MB/s 
     |████████████████████████████████| 843 kB 54.5 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
     |████████████████████████████████| 112 kB 64.5 MB/s 
  Created wheel for detectron2: filename=detectron2-0.6-cp37-cp37m-linux_x86_64.whl size=5722778 sha256=24b64dd4b9c6caf6b2659b3961170d4e59966625872dc986ca186ffb0c79c4f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-dplbddcx/wheels/07/dc/32/0322cb484dbefab8b9366bfedbaff5060ac7d149d69c27ca5d
  Created wheel for fvcore: filename=fvcore

In [ ]:
IPATH = '/content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls'
!pip install {IPATH}/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links /content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls
!pip install {IPATH}/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links /content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls
!pip install   {IPATH}/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links /content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls
!pip install {IPATH}/detectron2-0.5/detectron2 --no-index --find-links /content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls 

In [8]:
import sys
sys.path.append('/content/drive/MyDrive/Kaggle/Sartorius')

In [9]:
import torch
from detectron2 import model_zoo
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
import cv2
import os
import pycocotools.mask as mask_util
import numpy as np
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from typing import Any, Iterator, List, Union
import numpy as np
import detectron2
from detectron2.engine import DefaultTrainer
from types import SimpleNamespace
from detectron2.structures.masks import ROIMasks
from detectron2.layers.mask_ops import paste_masks_in_image
from detectron2.evaluation import DatasetEvaluators
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.evaluation import inference_on_dataset
from detectron2.checkpoint import DetectionCheckpointer
from scipy.ndimage.morphology import binary_fill_holes
from skimage.morphology import dilation, erosion
from skimage.measure import label
from utils.post_processing import compute_overlaps_masks
from fvcore.transforms import HFlipTransform, NoOpTransform

# Post Processing Config

In [28]:
class Config:

  THRESHOLDS = [.34646999695500497, .3134462119696301, .6716982906416183]#[0.2848161999265901,0.5901576776244156,0.6508687839015181]
  MIN_PIXELS = [60, 120, 60]#[60,80,50]
  PIXEL_CLASS_THRESHOLDS = [0.3, 0.25, 0.5]
  NMS_THRESH_TEST = 0.25

  anchor_generators_sizes = [[9], [17], [31], [64],[127]]
  anchor_generators_aspect_ratios = [[0.5, 1.0, 2.0]]

  pixel_mean = [128,128,128]
  pixel_std = [13.235,13.235,13.235]

# Custom TTA

需要custom一下tta的cnn类和transform 加入旋转

In [11]:
class CustomizedRCNNWithTTA(GeneralizedRCNNWithTTA):
  def _reduce_pred_masks(self, outputs, tfms):
    # Should apply inverse transforms on masks.
    # We assume only resize & flip are used. pred_masks is a scale-invariant
    # representation, so we handle flip specially
    for output, tfm in zip(outputs, tfms):
        if any(isinstance(t, HFlipTransform) for t in tfm.transforms):
            output.pred_masks = output.pred_masks.flip(dims=[3])
    all_pred_masks = [np.transpose(o.pred_masks.cpu().numpy().squeeze(),(1,2,0)) for o in outputs]
    # voting
    overlaps = compute_overlaps_masks(all_pred_masks[0],all_pred_masks[1])

    for mm in range(overlaps.shape[0]):
        if np.max(overlaps[mm]) > 0.1:
            ind = np.argmax(overlaps[mm])
            all_pred_masks[0][:, :, mm] = (all_pred_masks[0][:, :, mm] + all_pred_masks[1][:, :, ind])/2
            #all_pred_masks[0][:, :, mm] = (mask > 0).astype(np.uint8)
            #result1['scores'][mm] = 0.5*(result1['scores'][mm]+result2['scores'][ind])
        else:
            #print("Drop")
            all_pred_masks[0][:, :, mm] = 0
    all_pred_masks = [np.expand_dims(np.transpose(p,(2,0,1)),1) for p in all_pred_masks]
    return torch.tensor(all_pred_masks[0]).cuda()





# Utils

In [12]:
# def paste_masks_in_image(masks, boxes, image_shape, threshold=0.5):
#     """
#     Copy pasted from detectron2.layers.mask_ops.paste_masks_in_image and deleted thresholding of the mask
#     """
#     assert masks.shape[-1] == masks.shape[-2], "Only square mask predictions are supported"
#     N = len(masks)
#     if N == 0:
#         return masks.new_empty((0,) + image_shape, dtype=torch.uint8)
#     if not isinstance(boxes, torch.Tensor):
#         boxes = boxes.tensor
#     device = boxes.device
#     assert len(boxes) == N, boxes.shape

#     img_h, img_w = image_shape

#     # The actual implementation split the input into chunks,
#     # and paste them chunk by chunk.
#     if device.type == "cpu":
#         # CPU is most efficient when they are pasted one by one with skip_empty=True
#         # so that it performs minimal number of operations.
#         num_chunks = N
#     else:
#         # GPU benefits from parallelism for larger chunks, but may have memory issue
#         num_chunks = int(np.ceil(N * img_h * img_w * BYTES_PER_FLOAT / GPU_MEM_LIMIT))
#         assert (
#             num_chunks <= N
#         ), "Default GPU_MEM_LIMIT in mask_ops.py is too small; try increasing it"
#     chunks = torch.chunk(torch.arange(N, device=device), num_chunks)

#     img_masks = torch.zeros(
#         N, img_h, img_w, device=device, dtype=torch.float32
#     )
#     for inds in chunks:
#         masks_chunk, spatial_inds = _do_paste_mask(
#             masks[inds, None, :, :], boxes[inds], img_h, img_w, skip_empty=device.type == "cpu"
#         )
#         img_masks[(inds,) + spatial_inds] = masks_chunk
#     return img_masks

# detectron2.layers.mask_ops.paste_masks_in_image.__code__ = paste_masks_in_image.__code__

# def BitMasks__init__(self, tensor: Union[torch.Tensor, np.ndarray]):
#   """
#   Args:
#   tensor: bool Tensor of N,H,W, representing N instances in the image.
#   """
#   device = tensor.device if isinstance(tensor, torch.Tensor) else torch.device("cpu")
#   tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device) # Original code: tensor = torch.as_tensor(tensor, dtype=torch.bool, device=device)
#   assert tensor.dim() == 3, tensor.size()
#   self.image_size = tensor.shape[1:]
#   self.tensor = tensor

# detectron2.structures.masks.BitMasks.__init__.__code__ = BitMasks__init__.__code__

In [13]:
# def check_overlap(msk):
#     msk = np.array(msk)
#     msk = msk.astype(np.bool).astype(np.uint8)
#     return np.any(np.sum(msk, axis=-1)>1)

# def fix_overlap(msk):
#     """
#     Args:
#         mask: multi-channel mask, each channel is an instance of cell, shape:(520,704,None)
#     Returns:
#         multi-channel mask with non-overlapping values, shape:(520,704,None)
#     """

#     msk = np.transpose(msk,(1,2,0))
#     msk = np.pad(msk, [[0,0],[0,0],[1,0]])
#     ins_len = msk.shape[-1]
#     msk = np.argmax(msk,axis=-1)
#     msk = tf.keras.utils.to_categorical(msk, num_classes=ins_len)
#     msk = msk[...,1:]
#     msk = msk[...,np.any(msk, axis=(0,1))]
#     msk = np.transpose(msk,(2,0,1))
#     return msk


In [29]:
def precision_at(threshold, iou):
    matches = iou > threshold
    try:
      true_positives = np.sum(matches, axis=1) >= 1  # Correct objects
      # false_positives = np.sum(matches, axis=0) == 0  # Missed objects
      # false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
      false_positives = np.sum(matches, axis=1) == 0  # Extra objects
      false_negatives = np.sum(matches, axis=0) == 0  # Missed objects

      return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
    except:
      return 0,1,1


def score(pred, targ):
    enc_preds = post_process(pred)
    #enc_preds = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in enc_preds]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
        class_score[int(torch.mode(pred['instances'].pred_classes)[0])].append(p)
        #print("-"*50)
        #print(f"When threshold is {t}||Score is {p}||TP is {tp}||FP is {fp}||FN is {fn}")
    return np.mean(prec)

def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_
    rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return [x for x in runs]#' '.join(str(x) for x in runs)

# def get_masks(im,predictor):
#     pred = predictor([im]) 
#     pred_class = torch.mode(pred['instances'].pred_classes)[0]
#     # sreen class
#     take = pred['instances'].scores >= Config.THRESHOLDS[pred_class]
#     pred_masks = pred['instances'].pred_masks[take].cpu().numpy()
#     #pred_score = pred['instances'].scores[take].cpu().numpy()

#     # for p in pred_masks:
#     #     p = p.astype(np.uint8)
#     #     cont, hier = cv2.findContours(p,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     #     p = cv2.fillConvexPoly(np.zeros_like(p),points=cont[0], color=1)

#     res = []
#     used = np.zeros(im.shape[:2], dtype=int)
#     # pixel probability to mask
#     # THR = 0.5
#     #pred_masks = [np.where(tmp<=THR,0,tmp) for tmp in pred_masks]
#     # screen area 
#     for idx,mask in enumerate(pred_masks):
#         mask = mask * (1-used)
#         if mask.sum() >= Config.MIN_PIXELS[pred_class]: # skip predictions with small area
#             used += mask.astype(bool)
#             res.append(mask)
#     res = np.array(res)
#     # screen overlap
#     # if check_overlap(res):
#     #   return fix_overlap(res).astype(bool)
#     # else:
#     return res.astype(bool)

def post_process(pred):
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    # if pred_class == torch.tensor(1):
    #   print("Class is ", pred_class)
    # sreen class 
    take = pred['instances'].scores >= Config.THRESHOLDS[pred_class]
    pred_masks = pred['instances'].pred_masks[take].cpu().numpy()
    pred_score = pred['instances'].scores[take].cpu().numpy()
    #pred_masks = [pred_masks[idx]*pred_score[idx] for idx in range(len(pred_masks))]
    # print(np.max(pred_masks[0]))
    # print("-"*50)
    # print(pred_masks[-1])
    # print(np.max(pred_masks[-1]))
    # print("*"*50)
    # for p in pred_masks:
    #     p = p.astype(np.uint8)
    #     cont, hier = cv2.findContours(p,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #     p = cv2.fillConvexPoly(np.zeros_like(p),points=cont[0], color=1)

    res = []
    used = np.zeros(pred['instances'].pred_masks[0].shape)
    
    # pixel probability to mask
    # THR = 0.5
    #pred_masks = [np.array(np.where(tmp>Config.PIXEL_CLASS_THRESHOLDS[pred_class],1,0),dtype=np.uint8) for tmp in pred_masks]
    # screen area 


    for idx,mask in enumerate(pred_masks):
        mask = binary_fill_holes(mask).astype(np.uint8)
        mask = erosion(dilation(mask))

        if mask.sum() >= Config.MIN_PIXELS[pred_class]: # skip predictions with small area
            # Delete overlaps
            used += mask
            mask[used > 1] = 0
            out_label = label(mask)
            # Remove all the pieces if there are more than one pieces
            if out_label.max() > 1:
                mask[()] = 0
        # mask = mask * (1-used)
        # if mask.sum() >= Config.MIN_PIXELS[pred_class]: # skip predictions with small area
        #     used += mask
            res.append(mask.astype(bool))
    #res = np.array(res)
    # screen overlap
    # if check_overlap(res):
    #   return fix_overlap(res).astype(bool)
    # else:
    # print(len(res))
    # _, axs = plt.subplots(1,2, figsize=(40,15))
    # axs[1].imshow(cv2.imread("/content/drive/MyDrive/Kaggle/Sartorius/input/sartorius-cell-instance-segmentation/train/042dc0e561a4.png"))
    # for enc in res:
    #     axs[0].imshow(np.ma.masked_where(enc==0, enc))
    return res

# Generalized RCNN TTA

In [15]:
class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:

                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
   
        return {"MaP IoU": np.mean(self.scores)}

In [20]:
class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains pre-defined default logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can write your
    own training loop. You can use "tools/plain_train_net.py" as an example.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        #print(dataset_name)
        #return build_evaluator(cfg, dataset_name, output_folder)
        return MAPIOUEvaluator(dataset_name)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        #model = CustomizedRCNNWithTTA(cfg, model)
        #model = GeneralizedRCNNWithTTA(cfg, model)

        evaluator = [cls.build_evaluator(
            cfg, 'sartorius_val', # output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
        )]

        data_loader = cls.build_test_loader(cfg, 'sartorius_val')

        results_i = inference_on_dataset(model, data_loader, evaluator)
        return results_i

# Call

In [30]:
def run(model_config,model_path,val_path,):
  cfg = get_cfg()
  #cfg.merge_from_file(model_zoo.get_config_file(model_config))
  cfg.INPUT.MASK_FORMAT='bitmask'
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
  cfg.MODEL.WEIGHTS = model_path
  cfg.TEST.DETECTIONS_PER_IMAGE = 1000
  #cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 320
  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.OUTPUT_DIR = "output/"
  cfg.DATASETS.TEST = ('sartorius_val',)

  THR = 0.5

  #cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.25
  #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
  #cfg.MODEL.RPN.NMS_THRESH = 0.25
  #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
  #cfg.INPUT.MIN_SIZE_TEST = 800
  #cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 2000

  cfg.TEST.AUG.ENABLED = True
  cfg.TEST.AUG.MIN_SIZES = (1040,)
  cfg.TEST.AUG.FLIP = True
  #############################################################################
  # Cascade Team
  cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml"))
  cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Kaggle/Sartorius/model/TEAM/cascade_cv316/1218_cv316.pth"#这个是用livecell迭代6503次的预训练权重
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.INPUT.MIN_SIZE_TEST = 1040
  cfg.INPUT.MAX_SIZE_TEST = 1408
  cfg.SOLVER.AMP.ENABLED  = False      
  #cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
  #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
  # cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch
  # cfg.SOLVER.CHECKPOINT_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH 
  #############################################################################
  # Cascade Allin
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2","p2", "p3", "p4", "p5", "p6"]
  # cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25,0.5, 1.0, 2.0,4.0]]
  # cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[4],[9], [17], [31], [64],[127]]
  # cfg.MODEL.PIXEL_MEAN = [128,128,128]
  # cfg.MODEL.PIXEL_STD = [13.235,13.235,13.235]

  #############################################################################
  # Cascade vX
  # cfg.MODEL.PIXEL_MEAN = Config.pixel_mean
  # cfg.MODEL.PIXEL_STD = Config.pixel_std
  # cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]
  # cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[9], [17], [31], [64],[127]]
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]
  # cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 3000
  # cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 3000
  #############################################################################
  # Cascade v0
  # cfg.MODEL.RESNETS.DEPTH = 101
  # cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]
  # cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[8], [16], [32], [64],[128]]
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5","p6"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]

  # cfg.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, True, True, True] # on Res3,Res4,Res5
  # cfg.MODEL.RESNETS.DEFORM_MODULATED = True
  # cfg.MODEL.RESNETS.DEFORM_NUM_GROUPS = 2
  # cfg.MODEL.RESNETS.NORM = "GN"
  # cfg.MODEL.ROI_BOX_HEAD.NAME = "FastRCNNConvFCHead"
  # cfg.MODEL.ROI_BOX_HEAD.NUM_CONV = 4
  # cfg.MODEL.ROI_BOX_HEAD.NUM_FC = 1
  # cfg.MODEL.ROI_BOX_HEAD.NORM = "GN"  
  # cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION = 7
  # cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = True  
  # cfg.MODEL.ROI_MASK_HEAD.NUM_CONV = 8
  # cfg.MODEL.ROI_MASK_HEAD.NORM = "GN"

  # cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 3000
  # cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 3000
  #############################################################################

  cfg.freeze()
  
  DatasetCatalog.clear()
  MetadataCatalog.clear()

  register_coco_instances('sartorius_val',{},val_path, 
                          '/content/drive/MyDrive/Kaggle/Sartorius/input/sartorius-cell-instance-segmentation')

  val_ds = DatasetCatalog.get('sartorius_val')

  model = Trainer.build_model(cfg)

  DetectionCheckpointer(model, save_dir=model_path).resume_or_load(
      cfg.MODEL.WEIGHTS, resume=False
  )

  scr = Trainer.test_with_TTA(cfg, model)

  return scr["MaP IoU"]


In [22]:
model_path = "/content/drive/MyDrive/Kaggle/Sartorius/model/mask_rcnn_allin/fast_fold1/model_best.pth"#"/content/drive/MyDrive/Kaggle/Sartorius/model/finetuned/cascade_v0/fold_1/model_best.pth"#'/content/drive/MyDrive/Kaggle/Sartorius/model/finetuned/cascade_v0/fold_2/model_best.pth' 
model_config = "Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml" ##"Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
val_path = "/content/drive/MyDrive/Kaggle/Sartorius/input/all/annotations_val.json"#/content/drive/MyDrive/Kaggle/Sartorius/input/fold/coco_cell_valid_fold1.json"

In [4]:
import re
for fold in range(1,6):
  
  class_score = {0:[],1:[],2:[]}
  print("*"*50)
  print(f"Fold: {fold}")
  #run(model_config,model_path,re.sub("fold[0-9]",f"fold{fold}",val_path))
  print(run(model_config,re.sub(r"_[0-9]",f"_{fold}",model_path),re.sub("fold[0-9]",f"fold{fold}",val_path)))
  for i in range(3):
    print(f"Class {i} is ",np.mean(class_score[i]))

**************************************************
Fold: 1


NameError: ignored

In [31]:
class_score = {0:[],1:[],2:[]}
run(model_config,model_path,val_path)

0.30476707091765803

0.25+myThr: 0.304

0.3+myThr: 0.3031593457462934

0.25+DefaultThr: 0.3059191409796221

0.25+DefaultThr+FillHole: 0.30607070778380524

0.25+DefaultThr+DefaultLoop: 
0.3074941765053574

0.25+DefaultThr+DefaultLoop+FillHole: 
0.3074941765053574

All Default: 0.3081650706541967

All Default+Flip TTA: 0.3100357583342961

Tuned THr: 0.3098967254612715[链接文字](https://)

Tuned Thr+Flip TTA: 0.31113196842864865

800*800: 0.2948983500698248

R-50
**************************************************
Fold: 1
Class 0 is  0.16532477337361504
Class 1 is  0.1661711773848045
Class 2 is  0.4151430254951384
**************************************************
Fold: 2
Class 0 is  0.1631561086639971
Class 1 is  0.19641715950974975
Class 2 is  0.44511429321092794
**************************************************
Fold: 3
Class 0 is  0.16101907667878318
Class 1 is  0.19916202332004998
Class 2 is  0.41965532806598116
**************************************************
Fold: 4
Class 0 is  0.16053229469662908
Class 1 is  0.22732299168697404
Class 2 is  0.44011510995575753
**************************************************
Fold: 5
Class 0 is  0.16389088719357905
Class 1 is  0.1778480299102081
Class 2 is  0.4212842891027414

In [ ]:
0.37750409891510284

Voting: 37.73
Naive: 38.123 (800)
       37.73  (760,800,840) 

In [ ]:
for i in range(3):
  print(f"Class {i} is ",np.mean(class_score[i]))

Class 0 is  0.1631561086639971
Class 1 is  0.19641715950974975
Class 2 is  0.44511429321092794


In [ ]:
# 800
Class 0 is  0.18305352862872185
Class 1 is  0.2995814545992504
Class 2 is  0.4768256834723248

# 700
Class 0 is  0.18336778996390374
Class 1 is  0.30463719646665577
Class 2 is  0.48045191344321925

# 660 
Class 0 is  0.17938878020483373
Class 1 is  0.3114738942768526
Class 2 is  0.4791516489424115

# 520
Class 0 is  0.15878018976165423
Class 1 is  0.2839313849195593
Class 2 is  0.4624626261593129

## CASCADE
Class 0 is  0.16682632408412154

Class 1 is  0.13539552877767735

Class 2 is  0.4030932406202149

## MASK

Class 0 is  0.1407098304473609

Class 1 is  0.12648556317075743

Class 2 is  0.3725705952721068

## XL MASK
# fold1

Class 0 is  0.12059573687689787

Class 1 is  0.35384126190662685

Class 2 is  0.1218436842893815

# fold2

Class 0 is  0.09843779977582012

Class 1 is  0.11107452698340003

Class 2 is  0.3867325669552881

# Simple Mask

fold 3

Class 0 is  0.13267437773681637

Class 1 is  0.18588357090782615

Class 2 is  0.39239618507202784

In [ ]:
# [I 2021-12-08 03:57:41,747] Trial 10 finished with value: 0.28539669587261673 and parameters: {'threshold_class_1': 0.10740575667523036, 'threshold_class_2': 0.3478905779744579, 'threshold_class_3': 0.6678254093148331, 'min_pixel_class_1': 100, 'min_pixel_class_2': 120, 'min_pixel_class_3': 60, 'threshold_NMS': 0.8876804104593865}. Best is trial 10 with value: 0.28539669587261673.


## Visualization

In [ ]:
import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog
import cv2
import pycocotools.mask as mask_util
import numpy as np
import sys
sys.path.append("/content/drive/MyDrive/Kaggle/Sartorius")

In [ ]:
from utils.sartorius_vis import show_predictions, show_zoomed

# Single Parameter Loop

In [ ]:
Config.PIXEL_CLASS_THRESHOLDS = [0.5,0.3,0.5]
pixel_scr = []

for x in np.arange(0.4,6,0.05):
  Config.PIXEL_CLASS_THRESHOLDS[0] = float(x)
  pixel_scr.append(run(model_config,model_path,val_path))

plt.figure()
plt.plot(np.arange(0.1,1,0.1),pixel_scr)


# OPTUNA

In [24]:
!pip install optuna

     |████████████████████████████████| 308 kB 12.7 MB/s 
     |████████████████████████████████| 209 kB 60.4 MB/s 
     |████████████████████████████████| 80 kB 10.0 MB/s 
     |████████████████████████████████| 75 kB 4.6 MB/s 
     |████████████████████████████████| 149 kB 57.5 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 112 kB 66.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=3738f0b9cd3fac76ecf9ec64cba1ce31c75ffd8cd3ca5d95fc6b11c0831acd33
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [25]:
import warnings

warnings.simplefilter("ignore", UserWarning)

In [26]:
import optuna
from optuna.samplers import TPESampler
# parameters search
class params_optim():
  def __init__(self,train_time):
    self.train_time = train_time
  
  def search(self):

    study = optuna.create_study(study_name="Post_Processing_Parameters",direction="maximize",sampler=TPESampler())
    study.optimize(self.objective,n_trials=self.train_time)
    best_trial = study.trial
    print(best_trial)
    return best_trial.params

  def objective(self,trial):
    params = {"threshold_class_1":trial.suggest_uniform("threshold_class_1",0.1,0.5),
              "threshold_class_2":trial.suggest_uniform("threshold_class_2",0.3,0.7),
              "threshold_class_3":trial.suggest_uniform("threshold_class_3",0.4,0.8),
              #"threshold_score_test":trial.suggest_uniform("threshold_score_test",0,1),
              "min_pixel_class_1":trial.suggest_int("min_pixel_class_1",30,100,10),
              "min_pixel_class_2":trial.suggest_int("min_pixel_class_2",40,200,20),
              "min_pixel_class_3":trial.suggest_int("min_pixel_class_3",30,100,10),
              #'threshold_NMS':trial.suggest_uniform('threshold_NMS',0.1,0.9),
              #"threshold_pixel_class_1":trial.suggest_uniform("threshold_pixel_class_1",0.1,0.9),
              #"threshold_pixel_class_2":trial.suggest_uniform("threshold_pixel_class_2",0.1,0.9),
              #"threshold_pixel_class_3":trial.suggest_uniform("threshold_pixel_class_3",0.1,0.9),


              }

    # cfg = get_cfg()
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
    # cfg.INPUT.MASK_FORMAT='bitmask'
    # cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
    # cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/Kaggle/Sartorius/model/model_final.pth'  
    # cfg.TEST.DETECTIONS_PER_IMAGE = 1000
    # cfg.DATALOADER.NUM_WORKERS = 2
    # cfg.OUTPUT_DIR = "output/"
    # cfg.DATASETS.TEST = ('sartorius_val',)

    # cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.3
    # cfg.TEST.AUG.ENABLED = True
    # cfg.TEST.AUG.MIN_SIZES = (760, 800, 840)
    # cfg.TEST.AUG.MAX_SIZE = 1400
    # cfg.TEST.AUG.FLIP = True

    # cfg.freeze()
    # DatasetCatalog.clear()
    # MetadataCatalog.clear()

    # register_coco_instances('sartorius_val',{},'/content/drive/MyDrive/Kaggle/Sartorius/input/all/annotations_val.json', 
    #                         '/content/drive/MyDrive/Kaggle/Sartorius/input/sartorius-cell-instance-segmentation')

    # val_ds = DatasetCatalog.get('sartorius_val')


    Config.THRESHOLDS = [params["threshold_class_1"], params["threshold_class_2"], params["threshold_class_3"]]
    Config.MIN_PIXELS = [params["min_pixel_class_1"], params["min_pixel_class_2"], params["min_pixel_class_3"]] # shsy5y, astro, cort
    #Config.NMS_THRESH_TEST = params['threshold_NMS']
    #Config.MIN_PIXELS = [75,150,75]
    #Config.PIXEL_CLASS_THRESHOLDS = [params["threshold_pixel_class_1"], params["threshold_pixel_class_2"], params["threshold_pixel_class_3"]]

    # model = Trainer.build_model(cfg)

    # DetectionCheckpointer(model, save_dir="/content/drive/MyDrive/Kaggle/Sartorius/model/model_final.pth").resume_or_load(
    #     cfg.MODEL.WEIGHTS, resume=False
    # )
    # # res = Trainer.test(cfg, model)
    # # if cfg.TEST.AUG.ENABLED:
    # #     res.update(Trainer.test_with_TTA(cfg, model))

    # scr = Trainer.test_with_TTA(cfg, model)["MaP IoU"]

    scr = run(model_config,model_path,val_path)

    return scr
  

In [27]:
params = params_optim(120).search()

[I 2021-12-17 18:05:31,550] A new study created in memory with name: Post_Processing_Parameters
[I 2021-12-17 18:07:25,088] Trial 0 finished with value: 0.29985320040126795 and parameters: {'threshold_class_1': 0.2737537711207999, 'threshold_class_2': 0.6103176446882081, 'threshold_class_3': 0.778741989808098, 'min_pixel_class_1': 50, 'min_pixel_class_2': 80, 'min_pixel_class_3': 50}. Best is trial 0 with value: 0.29985320040126795.
[I 2021-12-17 18:09:16,918] Trial 1 finished with value: 0.3039527807589388 and parameters: {'threshold_class_1': 0.39900127191359036, 'threshold_class_2': 0.3930852956354255, 'threshold_class_3': 0.7434845009692734, 'min_pixel_class_1': 100, 'min_pixel_class_2': 120, 'min_pixel_class_3': 80}. Best is trial 1 with value: 0.3039527807589388.
[I 2021-12-17 18:11:10,332] Trial 2 finished with value: 0.3037319098922834 and parameters: {'threshold_class_1': 0.2929385305399766, 'threshold_class_2': 0.5615343361735181, 'threshold_class_3': 0.41161236452533667, 'mi

AttributeError: ignored

fold 1#
[I 2021-11-30 07:14:14,108] Trial 81 finished with value: 0.264550685879924 and parameters: {'threshold_class_1': 0.16274521287624244, 'threshold_class_2': 0.6053078974405149, 'threshold_class_3': 0.35245807017059005, 'min_pixel_class_1': 70, 'min_pixel_class_2': 40, 'min_pixel_class_3': 50}. Best is trial 81 with value: 0.264550685879924.


# POST PROCESS
[I 2021-12-01 23:35:49,548] Trial 79 finished with value: 0.29081823463426554 and parameters: {'threshold_class_1': 0.3053350974825099, 'threshold_class_2': 0.4348695941397964, 'threshold_class_3': 0.835716430468768, 'min_pixel_class_1': 30, 'min_pixel_class_2': 160, 'min_pixel_class_3': 60, 'threshold_NMS': 0.3168324479695911}. Best is trial 79 with value: 0.29081823463426554.


# Conclusion

1. 重合的直接clip效果比重合的拼接好

